In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

In [ ]:
base_dir = '/content/drive/MyDrive/yolo_project'
model_path = os.path.join(base_dir, 'best.pt')
images_dir = os.path.join(base_dir, 'images')
videos_dir = os.path.join(base_dir, 'videos')

In [ ]:
print("YOLO 모델:", model_path)
print("이미지 파일:", os.listdir(images_dir))
print("영상 파일:", os.listdir(videos_dir))

In [ ]:
!pip install ultralytics --quiet

In [ ]:
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO(model_path)

# 5️⃣ 이미지 예측
sample_image = os.path.join(images_dir, 'img1.jpg')
results = model.predict(source=sample_image, show=True)

# 6️⃣ 비디오 예측
sample_video = os.path.join(videos_dir, 'vid1.mp4')
results_video = model.predict(source=sample_video, show=True)